In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

#prepara the data
train_path = "../input/train/train"
test_path = "../input/test/test"

label_frame = pd.read_csv('../input/train.csv')
test_frame = pd.read_csv('../input/sample_submission.csv')
x_train = []
x_test = []
y_train = np.array(label_frame['has_cactus'])
#load images
from keras.preprocessing import image
for fname in label_frame['id']:
    image_path = os.path.join(train_path , fname)
    pil_image = image.load_img(image_path, target_size=(32, 32, 3))
    np_image = image.img_to_array(pil_image)
    x_train.append(np_image)
for fname in test_frame['id']:
    image_path = os.path.join(test_path,fname)
    pil_image = image.load_img(image_path,target_size = (32,32,3))
    np_image = image.img_to_array(pil_image)
    x_test.append(np_image)
#trans to array
x_train = np.array(x_train)
x_train = x_train.astype('float32')/255
x_test = np.array(x_test)
x_test = x_test.astype('float32')/255
print(x_train.shape)
print(x_test.shape)

Using TensorFlow backend.


(17500, 32, 32, 3)
(4000, 32, 32, 3)


In [2]:
#build model
from keras.applications import VGG16
from keras import models
from keras import layers
from keras import optimizers
from keras.callbacks import EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
def get_model():
    base = VGG16(include_top = False,weights = 'imagenet',input_shape = (32,32,3))
    base.trainable = True
    base.summary()
    set_trainable = False
    for layer in base.layers:
        if layer.name == 'block5_conv3':
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False
    model = models.Sequential()
    model.add(base)
    model.add(layers.Flatten())
    
    model.add(layers.BatchNormalization())
    
    model.add(layers.Dense(256,activation = 'relu'))
    
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1,activation = 'sigmoid'))
    model.summary()
    model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['acc'])
    return model
    
model = get_model()
'''
x_val = x_train[16600:]
y_val = y_train[16600:]

model_check_point = ModelCheckpoint('./model.h5',monitor = 'val_loss',save_best_only = True)
early_stopping = EarlyStopping(monitor = 'val_loss',patience = 25)
reduce_lr_on_plateau = ReduceLROnPlateau(monitor = 'val_loss',patience = 15)

history = model.fit(x_train[:16600],y_train[:16600],epochs = 80,batch_size = 250,validation_data = (x_val,y_val),\
        callbacks = [model_check_point,reduce_lr_on_plateau])
#visualize
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()
'''

Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 3s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_____

"\nx_val = x_train[16600:]\ny_val = y_train[16600:]\n\nmodel_check_point = ModelCheckpoint('./model.h5',monitor = 'val_loss',save_best_only = True)\nearly_stopping = EarlyStopping(monitor = 'val_loss',patience = 25)\nreduce_lr_on_plateau = ReduceLROnPlateau(monitor = 'val_loss',patience = 15)\n\nhistory = model.fit(x_train[:16600],y_train[:16600],epochs = 80,batch_size = 250,validation_data = (x_val,y_val),        callbacks = [model_check_point,reduce_lr_on_plateau])\n#visualize\nimport matplotlib.pyplot as plt\n\nacc = history.history['acc']\nval_acc = history.history['val_acc']\nloss = history.history['loss']\nval_loss = history.history['val_loss']\n\nepochs = range(len(acc))\n\nplt.plot(epochs, acc, 'bo', label='Training acc')\nplt.plot(epochs, val_acc, 'b', label='Validation acc')\nplt.title('Training and validation accuracy')\nplt.legend()\n\nplt.figure()\n\nplt.plot(epochs, loss, 'bo', label='Training loss')\nplt.plot(epochs, val_loss, 'b', label='Validation loss')\nplt.title('Tr

In [3]:
model.fit(x_train,y_train,epochs = 100,batch_size = 250)
y_predictions = model.predict(x_test)
result = pd.DataFrame({'id' : pd.read_csv('../input/sample_submission.csv')['id'],'has_cactus' : y_predictions.squeeze()})
result.to_csv("submission.csv", index=False, columns=['id', 'has_cactus'])
print('submit successful')

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
17500/17500 [==============================] - 5s 311us/step - loss: 0.1587 - acc: 0.9459
Epoch 2/100
17500/17500 [==============================] - 2s 109us/step - loss: 0.0740 - acc: 0.9765
Epoch 3/100
17500/17500 [==============================] - 2s 109us/step - loss: 0.0692 - acc: 0.9781
Epoch 4/100
17500/17500 [==============================] - 2s 112us/step - loss: 0.0486 - acc: 0.9840
Epoch 5/100
17500/17500 [==============================] - 2s 110us/step - loss: 0.0462 - acc: 0.9841
Epoch 6/100
17500/17500 [==============================] - 2s 112us/step - loss: 0.0423 - acc: 0.9863
Epoch 7/100
17500/17500 [==============================] - 2s 109us/step - loss: 0.0406 - acc: 0.9868
Epoch 8/100
17500/17500 [==============================] - 2s 109us/step - loss: 0.0350 - acc: 0.9875
Epoch 9/100
17500/17500 [==============================] - 2s 110us/step - loss: 0.0344 - acc: 0.9887
Epoch 10/100
17500/17500 [========

KeyboardInterrupt: 